In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from PIL import Image

from keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from keras.models import Sequential
from keras.utils import to_categorical

import os

import random
random.seed(0)


In [2]:
PATH = 'notMNIST_small'

classes = os.listdir(PATH)
num_classes = len(classes)

print("There are {} classes: {}".format(num_classes, classes))

There are 10 classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']


In [26]:
X = []
y = []

for directory in os.listdir(PATH):
    for image in os.listdir(PATH + '/' + directory):
        try:
            path = PATH + '/' + directory + '/' + image
            img = Image.open(path)
            img.load()
            img_X = np.asarray(img, dtype=np.int16)
            X.append(img_X)
            y.append(directory)
        except:
            None
            
X = np.asarray(X)
y = np.asarray(y)

num_images = len(X)
size = len(X[0])

y = list(map(lambda x: ord(x) - ord('A'), y))
y = np.asarray(y)

indices = np.arange(X.shape[0])
np.random.shuffle(indices)

X = X[indices]
y = y[indices]

num_train_img = np.int64(0.7 * X.shape[0])

X_train = X[:num_train_img]
y_train = y[:num_train_img]

X_test = X[num_train_img:]
y_test = y[num_train_img:]

if y_train.ndim == 1: y_train = to_categorical(y_train, 10)
if y_test.ndim == 1: y_test = to_categorical(y_test, 10)

if np.max(X_train) == 255: X_train = X_train / 255 
if np.max(X_test) == 255: X_test = X_test / 255

if X_train.ndim == 3:
    num_pixels = size * size
    X_train_1d = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
    X_test_1d = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')

if X_train.ndim == 3:
    X_train = X_train.reshape(-1, size, size, 1).astype('float32')
    X_test = X_test.reshape(-1, size, size, 1).astype('float32')

input_shape = X_train.shape[1:]


input_shape_1d = X_train_1d.shape[1]


In [27]:
model_mlp = Sequential([
    Dense(512, input_dim=input_shape_1d, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
    
])

model_mlp.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')
model_mlp.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 512)               401920    
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dense_8 (Dense)             (None, 128)               32896     
                                                                 
 dense_9 (Dense)             (None, 10)                1290      
                                                                 
Total params: 567,434
Trainable params: 567,434
Non-trainable params: 0
_________________________________________________________________


In [28]:
model_mlp.fit(X_train_1d, y_train, epochs=10)


Epoch 1/10
410/410 [==============================] - 2s 4ms/step - loss: 0.4783 - accuracy: 0.8568
Epoch 2/10
410/410 [==============================] - 2s 5ms/step - loss: 0.3104 - accuracy: 0.9077
Epoch 3/10
410/410 [==============================] - 2s 5ms/step - loss: 0.2544 - accuracy: 0.9227
Epoch 4/10
410/410 [==============================] - 2s 6ms/step - loss: 0.2184 - accuracy: 0.9306
Epoch 5/10
410/410 [==============================] - 2s 5ms/step - loss: 0.1825 - accuracy: 0.9412
Epoch 6/10
410/410 [==============================] - 2s 4ms/step - loss: 0.1516 - accuracy: 0.9513
Epoch 7/10
410/410 [==============================] - 2s 5ms/step - loss: 0.1304 - accuracy: 0.9576
Epoch 8/10
410/410 [==============================] - 2s 5ms/step - loss: 0.1284 - accuracy: 0.9599
Epoch 9/10
410/410 [==============================] - 2s 4ms/step - loss: 0.1074 - accuracy: 0.9647
Epoch 10/10
410/410 [==============================] - 2s 4ms/step - loss: 0.0927 - accuracy: 0.9684

In [29]:
score = model_mlp.evaluate(X_test_1d, y_test, verbose=1)


176/176 [==============================] - 1s 3ms/step - loss: 0.3786 - accuracy: 0.9176


In [30]:
model_cnn = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    MaxPool2D(),
    
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    
    Flatten(),
    
    Dense(512, activation='relu'),
    Dense(num_classes, activation='softmax'),
])

model_cnn.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')
model_cnn.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1600)              0         
                                                                 
 dense_10 (Dense)            (None, 512)              

In [31]:
model_cnn.fit(X_train, y_train, epochs=10)


Epoch 1/10
410/410 [==============================] - 6s 13ms/step - loss: 0.5179 - accuracy: 0.8501
Epoch 2/10
410/410 [==============================] - 5s 13ms/step - loss: 0.2681 - accuracy: 0.9193
Epoch 3/10
410/410 [==============================] - 5s 12ms/step - loss: 0.1993 - accuracy: 0.9380
Epoch 4/10
410/410 [==============================] - 5s 12ms/step - loss: 0.1506 - accuracy: 0.9509
Epoch 5/10
410/410 [==============================] - 5s 12ms/step - loss: 0.1164 - accuracy: 0.9632
Epoch 6/10
410/410 [==============================] - 5s 12ms/step - loss: 0.0895 - accuracy: 0.9709
Epoch 7/10
410/410 [==============================] - 5s 12ms/step - loss: 0.0660 - accuracy: 0.9789
Epoch 8/10
410/410 [==============================] - 5s 12ms/step - loss: 0.0551 - accuracy: 0.9798
Epoch 9/10
410/410 [==============================] - 5s 12ms/step - loss: 0.0501 - accuracy: 0.9845
Epoch 10/10
410/410 [==============================] - 5s 12ms/step - loss: 0.0406 - accura

In [35]:
%%time
score = model_cnn.evaluate(X_test, y_test, verbose=1)


176/176 [==============================] - 1s 5ms/step - loss: 0.3424 - accuracy: 0.9320
CPU times: total: 5.95 s
Wall time: 935 ms
